<a href="https://colab.research.google.com/github/subhashpolisetti/Decision-Tree-Ensemble-Algorithms/blob/main/Decision_Tree_From_Scratch_Gini_Entropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Tree from Scratch: Gini Impurity and Entropy for Classification

This notebook demonstrates how to implement a decision tree from scratch for classification tasks. The decision tree uses **Gini Impurity** and **Entropy** as impurity measures to decide the best feature and threshold for splitting the dataset. The implementation follows the standard decision tree algorithm, where the goal is to minimize the impurity of the splits at each node.

## Key Concepts:

1. **Gini Impurity**: A measure used to determine how often a randomly chosen element would be incorrectly classified. The formula is `1 - sum(p_i^2)` where `p_i` is the probability of each class in the node.
   
2. **Entropy**: Another impurity measure, often used in decision trees like ID3. It quantifies the amount of disorder or uncertainty in the data. The formula is `-sum(p_i * log2(p_i))` where `p_i` is the probability of each class in the node.

3. **Information Gain**: This metric helps in choosing the best feature and threshold to split on. It calculates the reduction in impurity from the parent node to the child nodes after a split.

4. **Dataset**: In this notebook, we use the **Iris dataset** from `sklearn`, which is a well-known dataset used for classification tasks. It contains 150 samples of iris flowers from three different species, with four features each.

## Steps in the Notebook:

1. **Data Preprocessing**: The Iris dataset is loaded, and the data is split into training and testing sets (80% for training, 20% for testing).

2. **Building the Decision Tree**: The tree is built recursively using either **Gini Impurity** or **Entropy** to decide the best split at each node. The tree continues to split until it reaches the maximum depth or the stopping criteria are met (e.g., all samples belong to the same class or not enough samples to split further).

3. **Training**: The decision tree is trained on the training set, and predictions are made on the test set.

4. **Evaluation**: The accuracy of the model is computed using `accuracy_score` from `sklearn`.

5. **Tree Visualization**: The structure of the decision tree is printed to show the feature splits at each node and the class predictions at the leaf nodes.

## Libraries Used:
- **NumPy**: For numerical computations and array manipulations.
- **Counter**: From the `collections` module, used to count occurrences of each class in the target variable.
- **sklearn.datasets**: For loading the Iris dataset.
- **sklearn.model_selection**: For splitting the dataset into training and testing sets.
- **sklearn.metrics**: For evaluating the model's accuracy.

## Results:
- The decision tree is trained on the Iris dataset and achieves **100% accuracy** on the test set.
- The tree structure is printed, showing the feature splits and class predictions at the leaves.

Feel free to modify the hyperparameters like `max_depth` to see how the tree's depth affects performance. You can also experiment with other impurity functions like **Entropy** to compare the results.

---

This markdown description provides an overview of the decision tree algorithm, the dataset used, and the steps in the notebook. It also outlines the libraries and results, helping users understand the notebook's purpose and flow.


In [1]:

import numpy as np
from collections import Counter


In [2]:

def gini_impurity(y):
    counts = np.bincount(y)
    probabilities = counts / len(y)
    return 1 - np.sum(probabilities ** 2)

In [3]:

def entropy(y):
    counts = np.bincount(y)
    probabilities = counts / len(y)
    return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

In [5]:
def information_gain(y, y_left, y_right, impurity_func):
    p = len(y_left) / len(y)
    return impurity_func(y) - p * impurity_func(y_left) - (1 - p) * impurity_func(y_right)

In [6]:
def split_dataset(X, y, feature_index, threshold):
    left_indices = X[:, feature_index] <= threshold
    right_indices = X[:, feature_index] > threshold
    return X[left_indices], X[right_indices], y[left_indices], y[right_indices]

In [7]:

def find_best_split(X, y, impurity_func):
    best_gain = -1
    best_split = None
    n_features = X.shape[1]

    for feature_index in range(n_features):
        thresholds = np.unique(X[:, feature_index])
        for threshold in thresholds:
            X_left, X_right, y_left, y_right = split_dataset(X, y, feature_index, threshold)
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            gain = information_gain(y, y_left, y_right, impurity_func)
            if gain > best_gain:
                best_gain = gain
                best_split = (feature_index, threshold)
    return best_split

In [9]:
class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2, impurity_func=gini_impurity):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.impurity_func = impurity_func
        self.tree = None

    def fit(self, X, y, depth=0):
        if depth == self.max_depth or len(y) < self.min_samples_split or len(np.unique(y)) == 1:
            return Counter(y).most_common(1)[0][0]

        feature_index, threshold = find_best_split(X, y, self.impurity_func)
        if feature_index is None:
            return Counter(y).most_common(1)[0][0]

        X_left, X_right, y_left, y_right = split_dataset(X, y, feature_index, threshold)
        self.tree = {
            "feature_index": feature_index,
            "threshold": threshold,
            "left": self.fit(X_left, y_left, depth + 1),
            "right": self.fit(X_right, y_right, depth + 1)
        }
        return self.tree

    def predict_single(self, x, tree):
        if isinstance(tree, dict):
            if x[tree["feature_index"]] <= tree["threshold"]:
                return self.predict_single(x, tree["left"])
            else:
                return self.predict_single(x, tree["right"])
        return tree

    def predict(self, X):
        return np.array([self.predict_single(x, self.tree) for x in X])

In [10]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load dataset
data = load_iris()
X, y = data.data, data.target

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Decision Tree
tree = DecisionTree(max_depth=5)
tree.fit(X_train, y_train)

# Predict
y_pred = tree.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [11]:

def print_tree(tree, depth=0):
    if isinstance(tree, dict):
        print(f"{'|   ' * depth}Feature {tree['feature_index']} <= {tree['threshold']}")
        print_tree(tree["left"], depth + 1)
        print_tree(tree["right"], depth + 1)
    else:
        print(f"{'|   ' * depth}Predict: {tree}")

print_tree(tree.tree)

Feature 2 <= 1.9
|   Predict: 0
|   Feature 2 <= 4.7
|   |   Feature 3 <= 1.6
|   |   |   Predict: 1
|   |   |   Predict: 2
|   |   Feature 3 <= 1.7
|   |   |   Feature 2 <= 4.9
|   |   |   |   Predict: 1
|   |   |   |   Feature 3 <= 1.5
|   |   |   |   |   Predict: 2
|   |   |   |   |   Predict: 1
|   |   |   Feature 2 <= 4.8
|   |   |   |   Feature 0 <= 5.9
|   |   |   |   |   Predict: 1
|   |   |   |   |   Predict: 2
|   |   |   |   Predict: 2
